**Dev Software Tools App**

In this project, I will analyze develop and deploy a web application to a cloud service so that it is accessible to the public. I will explore various aspects of the data and perform different cleaning methods on the data to refine it into a more workable form.


In [1]:
# import all nesecarry packages
import pandas as pd
import streamlit as st
import plotly.express as px
import altair
import datetime as dt
import numpy as np

In [2]:
# read in the csv file
car_ad_data = pd.read_csv(r'C:\Users\Darth Piggyus\Project-software-dev-tools\vehicles_us.csv')

In [3]:
# create a new column for the maker by taking the first word in the model column
car_ad_data['maker'] = car_ad_data['model'].apply(lambda x:x.split()[0])
car_ad_data.sample()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,maker
34673,14500,2004.0,chevrolet silverado 2500hd,excellent,8.0,gas,106000.0,automatic,truck,grey,1.0,2018-09-04,36,chevrolet


In [4]:
# check for duplicated rows
print(car_ad_data.duplicated().sum())

0


In [5]:
# look at the data
car_ad_data.info()
car_ad_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
 13  maker         51525 non-null  object 
dtypes: float64(4), int64(2), object(8)
memory usage: 5.5+ MB


,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


In [6]:
# Group by 'maker' and 'model' and calculate median model year
median_model_year = car_ad_data.groupby(['maker', 'model'])['model_year'].median()

# Fill missing values in 'model_year' column with median values based on groups
car_ad_data['model_year'] = car_ad_data.apply(
    lambda row: median_model_year.loc[(row['maker'], row['model'])] if pd.isnull(row['model_year']) else row['model_year'],
    axis=1
)

# Group by 'model' and 'model_year' and calculate median number of cylinders
median_cylinders = car_ad_data.groupby(['model', 'model_year'])['cylinders'].median()

# Fill missing values in 'cylinders' column with median values based on groups
car_ad_data['cylinders'] = car_ad_data.apply(
    lambda row: median_cylinders.loc[(row['model'], row['model_year'])] if pd.isnull(row['cylinders']) else row['cylinders'],
    axis=1
)

# Calculate the average of the 'cylinders' column
average_cylinders = car_ad_data['cylinders'].mean()

# Fill missing values in the 'cylinders' column with the average
car_ad_data['cylinders'] = car_ad_data['cylinders'].fillna(average_cylinders)

# Group by 'model' and 'model_year' and calculate median number of odometer
median_odometer = car_ad_data.groupby(['model', 'model_year'])['odometer'].median()

# Fill missing values in 'odometer' column with median values based on groups
car_ad_data['odometer'] = car_ad_data.apply(
    lambda row: median_odometer.loc[(row['model'], row['model_year'])] if pd.isnull(row['odometer']) else row['odometer'],
    axis=1
)

# Calculate the average of the 'odometer' column
average_odometer = car_ad_data['odometer'].mean()

# Fill missing values in the 'odometer' column with the average
car_ad_data['odometer'] = car_ad_data['odometer'].fillna(average_odometer)

# Group by 'model' and 'model_year' and find the mode of 'paint_color'
mode_paint_color = car_ad_data.groupby(['model', 'model_year'])['paint_color'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

# Fill missing values in 'paint_color' column with mode values based on groups
car_ad_data['paint_color'] = car_ad_data.apply(
    lambda row: mode_paint_color.loc[(row['model'], row['model_year'])] if pd.isnull(row['paint_color']) else row['paint_color'],
    axis=1
)

# Calculate the average of the 'paint_color' column
average_color = car_ad_data['paint_color'].mode().iloc[0]

# Fill missing values in the 'paint_color' column with the average
car_ad_data['paint_color'] = car_ad_data['paint_color'].fillna(average_color)

# Group by 'model' and 'model_year' and find the mode of 'is_4wd'
mode_is_4wd = car_ad_data.groupby(['model', 'model_year'])['is_4wd'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

# Fill missing values in 'is_4wd' column with mode values based on groups
car_ad_data['is_4wd'] = car_ad_data.apply(
    lambda row: mode_is_4wd.loc[(row['model'], row['model_year'])] if pd.isnull(row['is_4wd']) else row['is_4wd'],
    axis=1
)

# Replace any remaining null values in 'is_4wd' column with 0
car_ad_data['is_4wd'] = car_ad_data['is_4wd'].fillna(0)


In [7]:
# fix the datatypes to make them easier to work with
car_ad_data = car_ad_data.astype({'price':'float','days_listed':'float'})
car_ad_data['date_posted'] = pd.to_datetime(car_ad_data['date_posted'])
car_ad_data['is_4wd'] = car_ad_data['is_4wd'].astype(bool)
car_ad_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  float64       
 1   model_year    51525 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      51525 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  bool          
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  float64       
 13  maker         51525 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(5), object(7)
memory usage: 5.2+ MB


In [8]:
# create a histogram comparing days listed on the market for 4WD and non-4WD vehicles
color_map = {True: 'red', False: 'blue'}
fig2 = px.histogram(car_ad_data, x='days_listed', color='is_4wd', 
                   labels={'days_listed': 'Days Listed', 'is_4wd': 'Vehicle Type'},
                   barmode='overlay', histnorm='', color_discrete_map=color_map, 
                   title='Histogram of Days Listed by 4WD')
fig2.for_each_trace(lambda t: t.update(name='4WD' if t.name == 'True' else 'Non-4WD'))
fig2.update_layout(yaxis_title='Number of Vehicles')




Here you can see there isn't really a large difference in the time it takes to sell a 4wd and non-4wd vehicle, even though there are far more 4wd vehicles listed.


In [9]:
# Group the car_ad_data by transmission type
transmission_grouped = car_ad_data.groupby('transmission').size().reset_index(name='count')

# Plot the data using Plotly Express
fig3 = px.bar(transmission_grouped, x='transmission', y='count',
             labels={'transmission': 'Transmission Type', 'count': 'Number of Vehicles'},
             title='Number of Vehicles by Transmission Type')
             
# Set x-axis title
fig3.update_xaxes(title='Transmission Type')

# Set y-axis title
fig3.update_yaxes(title='Number of Vehicles')

The vast majortiy of cars sold were automatic which suggests that peopole are happier with other transmission types or that far more automatic car are produced. 

In [10]:
# Group the data by 'model_year' and calculate the average price for each year
average_price_by_year = car_ad_data.groupby('model_year')['price'].mean().reset_index()

# Create a line graph to show the average price of cars sold each year
fig = px.line(average_price_by_year, x='model_year', y='price', 
              labels={'model_year': 'Year', 'price': 'Average Price'},
              title='Average Price of Cars Sold by Model Year')

# Set x-axis title
fig.update_xaxes(title='Year')

# Set y-axis title
# Extract month from 'date_posted' column and create a new column called 'month'
car_ad_data['month'] = car_ad_data['date_posted'].dt.month

# Display the first few rows of the updated DataFrame to verify the changes
st.write(car_ad_data.head())
fig.update_yaxes(title='Average Price')

2024-05-08 17:10:46.976 
  command:

    streamlit run C:\Users\Darth Piggyus\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


On average it appears cars from 1950-70 sell for a much higher price than other ears. However you do begin to see the price creep higher as the model year gets closer to the present.

In [11]:
# Extract month from 'date_posted' column and create a new column called 'month'
car_ad_data['month'] = car_ad_data['date_posted'].dt.month

color_palette = px.colors.qualitative.Alphabet[:19]

# Group the car_ad_data by maker and month
grouped_data = car_ad_data.groupby(['maker', 'month']).size().reset_index(name='count')

# Plot a histogram showing the number of cars listed by each maker each month
fig4 = px.histogram(grouped_data, x='month', y='count', color='maker',
                   labels={'count': 'Month', 'month': 'Month', 'maker': 'Car Maker'},
                   title='Number of Cars Listed by Each Maker Each Month',
                   barmode='group', nbins=12, color_discrete_sequence=color_palette)

# Set x-axis title
fig4.update_xaxes(title='Month')

# Set y-axis title
fig4.update_yaxes(title='Number of Listings')

This plot shows that Ford and Chevrolet far exceed other makers in the volume of used vehicles sold month after month.


**Conclusion**

Looking at the initial data it seems ford and chevy cars are the most popular and almost all of them are automatic 4wd vehicles. Listings throughout the year seem pretty consistant but drop noticably across the board in the 4th month.